In [1]:
import random
import numpy as np

https://www.notion.so/5ee5b31a2ed54fe88dee17a5fa64504c

In [28]:
D = ['D1','D2','D3','D4','D5']
P = ['P1','P2','P3','P4','P5']
hungarian(D,P)

주어진 D,P 간의 거리 행렬: 
 [[8 9 5 6 9]
 [9 6 1 1 7]
 [1 9 4 8 9]
 [5 9 3 6 2]
 [6 9 3 4 1]]
3-a: 
 [[3 4 0 1 4]
 [8 5 0 0 6]
 [0 8 3 7 8]
 [3 7 1 4 0]
 [5 8 2 3 0]]
3-b: 
 [[3 0 0 1 4]
 [8 1 0 0 6]
 [0 4 3 7 8]
 [3 3 1 4 0]
 [5 4 2 3 0]]
그래프 :
 [[1, 2], [2, 3], [0], [4], [4]]
역방향 그래프 :
 [[2], [0], [0, 1], [1], [3, 4]]
최대 간선 수: 4
간선 정보: [2, 0, 1, -1, 3]
매칭 정보: [(0, 1), (1, 2), (2, 0), (3, 4), (4, -1)]
line_num = 4, n = 5
3. 경로 4->9  |매칭선 --> 회색|
3. 경로 9->3  |회색 --> 매칭선|
3번 끝
9번 끝
4번 끝
C : [0 1 2 9]
최솟값 : 1
CO_row:  [0 1 2] CO_col:  [4]
Not_CO_row:  [3 4] Not_CO_col:  [0 1 2 3] 

CBG :
 [[3 0 0 1 4]
 [8 1 0 0 6]
 [0 4 3 7 8]
 [3 3 1 4 0]
 [5 4 2 3 0]]
CBG :5-b
 [[ 3  0  0  1  4]
 [ 8  1  0  0  6]
 [ 0  4  3  7  8]
 [ 2  2  0  3 -1]
 [ 4  3  1  2 -1]]
CBG :5-c
 [[3 0 0 1 5]
 [8 1 0 0 7]
 [0 4 3 7 9]
 [2 2 0 3 0]
 [4 3 1 2 0]]
그래프 :
 [[1, 2], [2, 3], [0], [2, 4], [4]]
역방향 그래프 :
 [[2], [0], [0, 1, 3], [1], [3, 4]]
최대 간선 수: 5
간선 정보: [2, 0, 3, 1, 4]
매칭 정보: [(0, 1), (1, 3), (2, 0), (3, 2), (4, 4)]
최

array([[8, 9, 5, 6, 9],
       [9, 6, 1, 1, 7],
       [1, 9, 4, 8, 9],
       [5, 9, 3, 6, 2],
       [6, 9, 3, 4, 1]])

In [19]:
# 노션 링크: https://www.notion.so/5ee5b31a2ed54fe88dee17a5fa64504c - 진행 과정
def hungarian(D, P): # D: Detections, P: Predictions
    
    CBG = makeCBG(D,P,10)
    origin_CBG = CBG.copy()
    CBG = subMin(CBG)    
    n = len(CBG)
    
    graph = makeGraph(CBG)
    graph_inv = makeGraph_inv(CBG)
    line_num, matching = findMM(graph)
    print('line_num = {}, n = {}'.format(line_num,n))
    while line_num < n:
        line = crossOut(graph, graph_inv, matching)
        CBG = AMM(CBG, line)
        
        graph = makeGraph(CBG)
        graph_inv = makeGraph_inv(CBG)
        line_num, matching = findMM(graph)        
    
    # mm[0] >= len(CBG)인 상태
    print('매칭 완료\n')
    matching = findMM(graph)[1]
    result = 0
    for (i,j) in matching:
        result += origin_CBG[i][j]
    print(result)
    
    return origin_CBG

In [20]:
# 노션 1, 2
# D, p 는 1차원 배열, threshold 지정

# D, P 간의 CBG를 Matrix 형태로 초기화한다
def makeCBG(D,P, threshold = 10): # CBG : Complete bipartite graph
    n = max(len(D),len(P))
    CBG = [[threshold for _ in range(n)] for _ in range(n)]
    
    for i in range(len(D)):
        for j in range(len(P)):            
            CBG[i][j] = getDistance(D[i],P[j]) # D,P 사이의 거리
            
    print('주어진 D,P 간의 거리 행렬: \n', np.array(CBG))
    return np.array(CBG)

def getDistance(d,p): # D, P 원소들의 거리(weight)
    # pass
    return random.randint(1,9)

In [21]:
# 노션 3

# 행렬의 행, 열을 돌면서 최솟값 빼주기
def subMin(M):     
    for row in range(len(M)):
        M[row] = M[row] - M[row].min()
    
    print('3-a: \n', M)
    for col in range(len(M)):
        M[:, col] = M[:, col] - M[:, col].min()
    print('3-b: \n', M)
    
    return M

In [22]:
# 노션 4

# 4-a
def makeGraph(CBG): # CBG는 사실 정방행렬(Square Matrix)이였는데 알고리즘 적용을 위해 Graph로 변환
    n = len(CBG)
    graph = []
    
    for row in range(n):
        temp = []
        for col in np.where(CBG[row]==0)[0]:
            temp.append(col)
        graph.append(temp)
    print('그래프 :\n',graph)
    return graph

# 4-b
def bimatch(N, graph, visited, selected): # 이분 매칭
    # print('bimatch!')
    if visited[N]:
        return False
    visited[N] = True
    
    for num in graph[N]:
        # print('{} -> {} 확인중'.format(N, num))
        # print('if문 이전:\n visited: {}\n selected: {}'.format(visited, selected))
        if selected[num] == -1 or bimatch(selected[num], graph, visited, selected):
            # print('{} -> {} 으로 대체'.format(N, num))
            selected[num] = N
            # print('if문 이후:\n visited: {}\n selected: {}'.format(visited, selected))
            return True
        
    return False

# 4-a,b
def findMM(Graph): # Find Maximum Matching
    n = len(Graph)
    selected = [-1] * (n)

    for i in range(n):
        visited = [False] * n
        bimatch(i, Graph, visited, selected)

    result = 0
    for i in range(n):
        if selected[i] >= 0:
            result += 1

    print('최대 간선 수: {}\n간선 정보: {}'.format(result, selected))
    matchings = []
    
    for idx in range(n):
        # assert idx not in mm[1], '매칭이 제대로 되지 않았다'
        try:
            matchings.append((idx, selected.index(idx)))
        except:
            matchings.append((idx, -1))
    print('매칭 정보: {}'.format(matchings))
    
    return result, matchings

In [24]:
# 노션 4-c

# 최소 버텍스 커버 문제
# 버텍스 커버문제에서 X를 찾기 위해 그래프의 통합이 필요하다
# alternating path : 최대 매칭에 포함되지 않는 노드에서 출발하여, 최대 매칭에 속하지 않는 엣지, 속하는 엣지를 '번갈아' 선택하는 경로이다

def makeGraph_inv(CBG): # 역방향 그래프
    n = len(CBG)
    graph_inv = []
    
    for col in range(n):
        temp = []
        for row in np.where(CBG[:,col]==0)[0]:
            temp.append(row)
        graph_inv.append(temp)
    print('역방향 그래프 :\n',graph_inv)
    return graph_inv

def inteGraph(graph, g_inv): # 좌우 정점 구분을 위해(R_V = R_V + n)
    for i in range(len(graph)):
        for j in range(len(graph[i])):
            graph[i][j] += len(graph)
    
    return graph + g_inv

def dfs(X, graph, start, visited, matchings, count=True): # alternating path 확인을 위한 matching, count
    
    if visited[start]==True:
        return False
    
    visited[start] = True
    
    for vertex in graph[start]:
        if not visited[vertex]:
            next_count = None
            flag = False
            if count == 0 and (([start,vertex] in matchings) or ([vertex,start] in matchings)): # alternationg path를 판별하기위해, 최대 매칭에 속한 엣지에 속할 경우 count 초기화
                print('3. 경로 {}->{}  |회색 --> 매칭선|'.format(start,vertex))
                next_count = not(count)
                flag = True
            elif count == 1 and (not([start,vertex] in matchings) and not([vertex,start] in matchings)):
                print('3. 경로 {}->{}  |매칭선 --> 회색|'.format(start,vertex))
                next_count = not(count)
                flag = True
            
            if flag:
                X.append(vertex)
                dfs(X, graph, vertex, visited, matchings, next_count)
                
    print("{}번 끝".format(start))
    
    return True

# 라인 정보 반환
def crossOut(graph, graph_inv, matchings): # 라인에 대한 정보를 정점 배열로 반환
    
    X = [] # L의 매칭되지 않은 정점들과, 그 정점으로부터 도달할 수 있는 l, R의 모든 정점...이 아님! alternating path !
    L_outsider = []
    
    # L에서 매칭되지 않은 정점 찾기&추가
    for L,R in matchings:
        if R == -1:
            L_outsider.append(L)
            X.append(L) # X에도 추가
    
    matchings = [[L,R+len(graph)] for [L,R] in matchings if R != -1] # 매칭되지 않은(R=-1) 매칭 지우기    
    # L_outsider을 시작으로 연결된 alternating path의 vertex를 모두 X에 넣어야 함 (dfs)    
    integraph = inteGraph(graph, graph_inv)
    
    
    for outsider in L_outsider: # X 구하기
        visited = [False] * len(integraph)
        dfs(X, integraph, outsider, visited, matchings, True)
    
    
    # alternating path 적용 해야함
    L = np.arange(len(graph))
    R = np.arange(len(graph),len(graph)*2)
    C = np.union1d(np.setdiff1d(L,X),np.intersect1d(R,X)) # len(graph) 보다 작은 것은 row, 큰 것은 col 라인이 된다    
    print('C :',C)
    return C            

In [25]:
# 노션 5

def AMM(CBG, line): # Augument Maximum Matching using 0-weight edges
    CBG_clone = CBG.copy()
    Not_CO_row = np.arange(len(CBG))
    Not_CO_col = np.arange(len(CBG))
    CO_row, CO_col = [], []
    
    for co in line: # co: cross out
        if co < len(CBG):
            CBG_clone[co] = 99 # 99를 co로 대체
            Not_CO_row = np.delete(Not_CO_row,np.where(Not_CO_row==co)) # cross out 아닌 행 만들기
            CO_row = np.append(CO_row, co)
            CO_row = CO_row.astype(int)
        else:
            CBG_clone[:,co%len(CBG)] = 99
            Not_CO_col = np.delete(Not_CO_col,np.where(Not_CO_col==co%len(CBG))) # cross out 아닌 행 만들기
            CO_col = np.append(CO_col, co%len(CBG))
            CO_col = CO_col.astype(int)
    # 5-a        
    minE = CBG_clone.min()
    print('최솟값 :', minE)
    print('CO_row: ', CO_row       , 'CO_col: ',CO_col)
    print('Not_CO_row: ', Not_CO_row,'Not_CO_col: ',Not_CO_col,'\n')
    
    # 5-b
    print('CBG :\n',CBG)
    for line in Not_CO_row:
        CBG[line] -= minE
    print('CBG :5-b\n',CBG)
    for line in CO_col:
        CBG[:,line] += minE
    print('CBG :5-c\n',CBG)
    
    return CBG

In [326]:
g = [[1], [0], [1, 3], [0, 3], [1, 2, 4]]
g_inv = [[1, 3], [0, 2, 4], [4], [2, 3], [4]]
mat = [[0, 1], [1, 0], [2, 3], [3, -1], [4, 2]]
line = crossOut(g, g_inv, mat)

3. 경로 3->5  |매칭선 --> 회색|
3. 경로 5->1  |회색 --> 매칭선|
1번 끝
5번 끝
3. 경로 3->8  |매칭선 --> 회색|
3. 경로 8->2  |회색 --> 매칭선|
3. 경로 2->6  |매칭선 --> 회색|
3. 경로 6->0  |회색 --> 매칭선|
0번 끝
6번 끝
2번 끝
8번 끝
3번 끝
C : [4 5 6 8]
